In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 32.1 MB/s eta 0:00:00


In [ ]:
from transformers import pipeline

# classifier = pipeline("zero-shot-classification") #Defaults to bart-large-mnli
bart_model = pipeline(model="facebook/bart-large-mnli")

In [ ]:
#sentences to test

foodOne = """ What do we have for dinner? """
foodTwo = """ I want to eat. """
foodThree = """ Can you help me choose a plate? """

helpOne = """ can you help me clean myself? """
helpTwo = """ I need help doing that! """

misc = """ I read the news today """
miscTwo = """ A crowd of people stood and stared """

In [ ]:
categories = ["Sentence said while eating", "Sentence to ask for help", "Miscelanous sentence"]

In [ ]:
bart_model(foodOne, categories)

{'sequence': ' What do we have for dinner? ',
 'labels': ['Sentence said while eating',
  'Miscelanous sentence',
  'Sentence to ask for help'],
 'scores': [0.8490579724311829, 0.0931280329823494, 0.05781395360827446]}

In [ ]:
bart_model(foodThree, categories)

{'sequence': ' Can you help me choose a plate? ',
 'labels': ['Sentence to ask for help',
  'Sentence said while eating',
  'Miscelanous sentence'],
 'scores': [0.9169546961784363, 0.050039201974868774, 0.03300616517663002]}

In [ ]:
bart_model(helpOne, categories)

{'sequence': ' can you help me clean myself? ',
 'labels': ['Sentence to ask for help',
  'Miscelanous sentence',
  'Sentence said while eating'],
 'scores': [0.9727047681808472, 0.025597060099244118, 0.001698226435109973]}

In [ ]:
bart_model(helpTwo, categories)

{'sequence': ' I need help doing that! ',
 'labels': ['Sentence to ask for help',
  'Miscelanous sentence',
  'Sentence said while eating'],
 'scores': [0.9730145931243896, 0.024756183847784996, 0.0022291846107691526]}

In [ ]:
bart_model(miscTwo, categories)

{'sequence': ' A crowd of people stood and stared ',
 'labels': ['Miscelanous sentence',
  'Sentence to ask for help',
  'Sentence said while eating'],
 'scores': [0.5375846028327942, 0.4068887531757355, 0.05552665889263153]}

In [ ]:
from transformers import pipeline, set_seed
generator = pipeline('text-generation', model='gpt2')
set_seed(42)
sentences = generator("Generate a senetence of someone asking for help in target? ", max_length= 25, num_return_sequences=5)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [ ]:
sentences


[{'generated_text': 'Generate a senetence of someone asking for help in target? \xa0Maybe you have. \xa0That information about'},
 {'generated_text': "Generate a senetence of someone asking for help in target? \xa0This may be your opponent's opponent before you"},
 {'generated_text': "Generate a senetence of someone asking for help in target? ???? Don't do it if you don't understand"},
 {'generated_text': 'Generate a senetence of someone asking for help in target? \xa0As I said, it appears pretty close.'},
 {'generated_text': 'Generate a senetence of someone asking for help in target? \xa0How can you know in advance that they have'}]

In [ ]:
print(sentences[1]['generated_text'])

how are you "

"Well, I don't like it then, and I didn


In [ ]:
#those are very simple booleans, in the actual application I thought we could use the google maps API to gather a string of the place the user is located and inject the place in the prompt to generate the sentence like:
# place = "Target"
# prompt = "generate a sentence of someone asking for help in (place)"

isAtHome = False
isAtRestaurant = True
isAtHospital = False

In [ ]:
#the prompts are the beggining of the sentences generated, but could be "can you generate a sentence that someone asking for help at home would say?" or something similar. I found that this model (gpt2.0) does not
#generate sentences well based of questions framed like previously but gpt 3.5 does that much better.

def generate_sentences(sentences_gen):
  if isAtHome:
    sentences_gen.extend(generator("can you help me going to ", max_length= 9, num_return_sequences=15))
    sentences_gen.extend(generator("can you bring me my ", max_length= 9, num_return_sequences=15))
  if isAtRestaurant:
    sentences_gen.extend(generator("can you help me order ", max_length= 8, num_return_sequences=15))
    sentences_gen.extend(generator("can I have the ", max_length= 7, num_return_sequences=15))
  if isAtHospital:
    sentences_gen.extend(generator("can you call ", max_length= 9, num_return_sequences=15))
    sentences_gen.extend(generator("Please bring me my ", max_length= 9, num_return_sequences=15))


In [ ]:
def generate_categories(catigories):
  if isAtHome:
    catigories.append("Sentence to ask for help at home")
  if isAtRestaurant:
    catigories.append("Sentence said while eating")
  if isAtHospital:
    catigories.append("Sentence to ask for doctor assistance")
  catigories.append("Sentence to ask for help")


In [ ]:
highest_scores = {category: 0.0 for category in categories}
highest_score_sentences = {category: "" for category in categories}

def classify_sentences(sentences, categories):
  for sentence in sentences:
    result = bart_model(sentence, categories)
    labels = result["labels"]
    scores = result["scores"]

    # Find the index of the category with the highest score
    max_score_index = scores.index(max(scores))

    # Check if the current sentence has a higher score than the previous highest score
    if scores[max_score_index] > highest_scores[labels[max_score_index]]:
        highest_scores[labels[max_score_index]] = scores[max_score_index]
        highest_score_sentences[labels[max_score_index]] = sentence
  for category, sentence in highest_score_sentences.items():
    print(f"Highest score in '{category}': {highest_scores[category]}")
    print(f"Sentence: {sentence}\n")

In [ ]:
gen_sentences = []
generate_sentences(gen_sentences)
generated_text_list = [item['generated_text'] for item in gen_sentences]
generated_text_list

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


['can you help me order �',
 'can you help me order ?????',
 'can you help me order ???? It',
 'can you help me order ____?',
 'can you help me order irl?"',
 'can you help me order \xa0a',
 'can you help me order !! I',
 'can you help me order "\n',
 'can you help me order ???? (',
 'can you help me order " he',
 'can you help me order ?????',
 'can you help me order �',
 'can you help me order ________?',
 'can you help me order !! It',
 'can you help me order iced coffee',
 'can I have the _______ thing',
 'can I have the ivan?',
 'can I have the _______,',
 'can I have the ileum',
 'can I have the ersatz',
 'can I have the iced tea',
 'can I have the izak',
 'can I have the _______________',
 'can I have the _______ on',
 'can I have the xtract',
 'can I have the \xa0part',
 'can I have the izatz',
 'can I have the iphone',
 'can I have the _______________',
 'can I have the ??????']

In [ ]:
gen_categories = []
generate_categories(gen_categories)
gen_categories

['Sentence said while eating', 'Sentence to ask for help']

In [ ]:
classify_sentences(generated_text_list, gen_categories)

Highest score in 'Sentence said while eating': 0.6769107580184937
Sentence: can I have the iced tea

Highest score in 'Sentence to ask for help': 0.9928860664367676
Sentence: can I have the iphone

Highest score in 'Miscelanous sentence': 0.0
Sentence: 



In [ ]:
def generate_recommended_sentences(generated_sen, gen_categories):
  generate_sentences(generated_sen)
  generated_text_list = [item['generated_text'] for item in generated_sen]
  generate_categories(gen_categories)
  classify_sentences(generated_text_list, gen_categories)


In [ ]:
generate_recommended_sentences([],[])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Highest score in 'Sentence said while eating': 0.6769107580184937
Sentence: can I have the iced tea

Highest score in 'Sentence to ask for help': 0.9928860664367676
Sentence: can I have the iphone

Highest score in 'Miscelanous sentence': 0.0
Sentence: 



In [ ]:
#could be used to generate prompts

def generate_prompt(base_prompt, user_location, position):
    if position > len(base_prompt) or position < 0:
        return "error"
    return base_prompt[:position] + user_location + base_prompt[position:]

In [ ]:
def create_generator(sentence, num_sentences, num_char):
  return generator(sentence, num_char, num_sentences)

In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 2.0 MB/s eta 0:00:00


In [ ]:
#If adopting gpt 3.5
import openai

num_sentences = 1 # Number of sentences to be generated
api_key = 'YOUR_API_KEY' # Replace 'YOUR_API_KEY' with your actual API key
openai.api_key = api_key
categories = ["Sentence said while eating", "Sentence to ask for help", "Miscelanous sentence"]

def generate_sentence(prompt):
    response = openai.Completion.create(
        engine="text-davinci-003",  # GPT-3.5 engine
        prompt=prompt,
        max_tokens=100,  # Adjust as needed to control the response length
        temperature=0.7,  # Controls the randomness of the output. Lower values make it more focused.
        stop=None,  # You can provide a stop token to limit the response if necessary.
        n=num_sentences,  # Number of responses to generate
        return_prompt=True,  # Returns the original prompt in the response object
    )

    return response.choices[0].text.strip()
